In [7]:
!nvidia-smi

Sun Sep 18 08:58:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
import json
token = {"username":"  ","key":"  "}#使用你自己的用户和密钥 Use your own username and key
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

- path is now set to: /content


In [9]:
!kaggle datasets download -d lzt2423654259/trashnet

 22% 9.00M/40.8M [00:00<00:00, 65.1MB/s]
100% 40.8M/40.8M [00:00<00:00, 181MB/s] 


In [10]:
!unzip '/content/datasets/lzt2423654259/trashnet/trashnet.zip'

Archive:  /content/datasets/lzt2423654259/trashnet/trashnet.zip
  inflating: dataset-resized/cardboard/cardboard1.jpg  
  inflating: dataset-resized/cardboard/cardboard10.jpg  
  inflating: dataset-resized/cardboard/cardboard100.jpg  
  inflating: dataset-resized/cardboard/cardboard101.jpg  
  inflating: dataset-resized/cardboard/cardboard102.jpg  
  inflating: dataset-resized/cardboard/cardboard103.jpg  
  inflating: dataset-resized/cardboard/cardboard104.jpg  
  inflating: dataset-resized/cardboard/cardboard105.jpg  
  inflating: dataset-resized/cardboard/cardboard106.jpg  
  inflating: dataset-resized/cardboard/cardboard107.jpg  
  inflating: dataset-resized/cardboard/cardboard108.jpg  
  inflating: dataset-resized/cardboard/cardboard109.jpg  
  inflating: dataset-resized/cardboard/cardboard11.jpg  
  inflating: dataset-resized/cardboard/cardboard110.jpg  
  inflating: dataset-resized/cardboard/cardboard111.jpg  
  inflating: dataset-resized/cardboard/cardboard112.jpg  
  inflating:

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
import time
import numpy as np
from torch.utils import data
from torchvision import transforms as T
from PIL import Image
import random
import os
import torchvision
from torch.autograd import Variable
from torch.optim import lr_scheduler

In [21]:
class Trash(data.Dataset):
    def __init__(self, root, transform=True, train=True, seed=1):
        self.labels = {'paper': 0, 'glass': 1, 'plastic': 2, 'cardboard': 3, 'trash': 4, 'metal': 5}
        dirs = os.listdir(root)  # 返回文件列表
        imgs = []  # 建立空列表
        for d in dirs:
            path = os.path.join(root, d)
            for img in os.listdir(path):
                imgs.append(os.path.join(path, img))
        self.train = train
        random.seed(seed)
        random.shuffle(imgs)
        num_train = int(0.7 * len(imgs))  # 训练集个数
        if train:
            # training
            self.imgs = imgs[:num_train]  # 设置训练集
        else:
            # validation
            self.imgs = imgs[num_train:]  #

        if transform is True:
            normalize = T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            if not train:
                # 验证集
                self.transform = T.Compose([
                    T.Resize(224),  # 最短边224像素
                    T.CenterCrop(224),  # 中心裁剪
                    T.ToTensor(),  # 转成tensor
                    normalize  # 归一化
                ])
            else:
                # 训练集，做数据增广
                self.transform = T.Compose([
                    T.Resize(224),
                    T.RandomResizedCrop(224),
                    T.RandomHorizontalFlip(),  # 按概率水平翻转
                    T.GaussianBlur(kernel_size=(3, 3)),
                    T.RandomAdjustSharpness(1.5),
                    T.ToTensor(),
                    normalize
                ])
        else:
            self.transform = transform

    def __getitem__(self, index):
        img_path = self.imgs[index]
        label = self.labels[img_path.split('/')[-2]]#linux 里的是/
        data = Image.open(img_path)
        data = torchvision.transforms.functional.adjust_gamma(data, 1.1)
        data = self.transform(data)
        return data, label

    def __len__(self):
        return len(self.imgs)

In [22]:
class CommonBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(CommonBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = x

        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        x = self.bn2(self.conv2(x))

        x += identity
        return F.relu(x, inplace=True)


class SpecialBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(SpecialBlock, self).__init__()
        self.change_channel = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride[0], padding=0, bias=False),
            nn.BatchNorm2d(out_channel)
        )
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride[0], padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=stride[1], padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)

    def forward(self, x):
        identity = self.change_channel(x)
        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        x = self.bn2(self.conv2(x))
        x += identity
        return F.relu(x, inplace=True)

class SelfAttention(nn.Module):
    """ Self attention Layer"""

    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.chanel_in = in_dim

        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 4, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 4, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)  #

    def forward(self, x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize, C, width, height = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width * height).permute(0, 2, 1)  # B X CX(N)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width * height)  # B X C x (*W*H)
        energy = torch.bmm(proj_query, proj_key)  # transpose check
        attention = self.softmax(energy)  # BX (N) X (N)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width * height)  # B X C X N
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, width, height)
        out = self.gamma * out + x
        return out
class SelfBlock(nn.Module):
    def __init__(self, in_channel, out_channel, stride):
        super(SelfBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.att1 = SelfAttention(in_channel)

    def forward(self, x):
        identity = x

        x = F.relu(self.bn1(self.conv1(x)), inplace=True)
        x = self.bn2(self.conv2(x))
        x = self.att1(x)
        x += identity
        return F.relu(x, inplace=True)

In [23]:
def Mask(nb_batch, channels):
    foo = [1]*1  + [0] * 1
    bar = []
    for i in range(256):
        random.shuffle(foo)
        bar += foo
    bar = [bar for i in range(nb_batch)]
    bar = np.array(bar).astype("float32")
    bar = bar.reshape(nb_batch, 512, 1, 1)
    bar = torch.from_numpy(bar)
    bar = bar.cuda()
    bar = Variable(bar)
    return bar


def supervisor(x, targets, height, cnum):
    mask = Mask(x.size(0), cnum)
    branch = x
    branch = branch.reshape(branch.size(0), branch.size(1), branch.size(2) * branch.size(3))
    branch = F.softmax(branch, 2)
    branch = branch.reshape(branch.size(0), branch.size(1), x.size(2), x.size(2))
    branch = my_MaxPool2d(kernel_size=(1, cnum), stride=(1, cnum))(branch)
    branch = branch.reshape(branch.size(0), branch.size(1), branch.size(2) * branch.size(3))
    loss_2 = 1.0 - 1.0 * torch.mean(torch.sum(branch, 2)) / cnum  # set margin = 3.0
    # print(x.size())
    # print(mask.size())
    branch_1 = x * mask
    # print(branch_1.size())
    branch_1 = my_MaxPool2d(kernel_size=(1, cnum), stride=(1, cnum))(branch_1)
    branch_1 = nn.AvgPool2d(kernel_size=(height, height))(branch_1)
    branch_1 = branch_1.view(branch_1.size(0), -1)
    criterion = nn.CrossEntropyLoss()
    loss_1 = criterion(branch_1, targets)

    return [loss_1, loss_2]



def position(H, W, is_cuda=True):
    if is_cuda:
        loc_w = torch.linspace(-1.0, 1.0, W).cuda().unsqueeze(0).repeat(H, 1)
        loc_h = torch.linspace(-1.0, 1.0, H).cuda().unsqueeze(1).repeat(1, W)
    else:
        loc_w = torch.linspace(-1.0, 1.0, W).unsqueeze(0).repeat(H, 1)
        loc_h = torch.linspace(-1.0, 1.0, H).unsqueeze(1).repeat(1, W)
    loc = torch.cat([loc_w.unsqueeze(0), loc_h.unsqueeze(0)], 0).unsqueeze(0)
    return loc

class my_MaxPool2d(nn.Module):

    def __init__(self, kernel_size, stride=None, padding=0, dilation=1,
                 return_indices=False, ceil_mode=False):
        super(my_MaxPool2d, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride or kernel_size
        self.padding = padding
        self.dilation = dilation
        self.return_indices = return_indices
        self.ceil_mode = ceil_mode

    def forward(self, input):
        input = input.transpose(3,1)
        input = F.max_pool2d(input, self.kernel_size, self.stride,
                            self.padding, self.dilation, self.ceil_mode,
                            self.return_indices)
        input = input.transpose(3,1).contiguous()

        return input

In [24]:
class ResNet34(nn.Module):
    def __init__(self, classes_num):
        super(ResNet34, self).__init__()
        self.prepare = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 1)
        )
        self.layer1 = nn.Sequential(
            CommonBlock(64, 64, 1),
            CommonBlock(64, 64, 1),
            CommonBlock(64, 64, 1)
        )

        self.layer2 = nn.Sequential(
            SpecialBlock(64, 128, [2, 1]),
            CommonBlock(128, 128, 1),
            CommonBlock(128, 128, 1),
            SelfBlock(128, 128, 1)
        )
        self.layer3 = nn.Sequential(
            SpecialBlock(128, 256, [2, 1]),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1),
            CommonBlock(256, 256, 1)
        )
        self.layer4 = nn.Sequential(
            SpecialBlock(256, 512, [2, 1]),
            CommonBlock(512, 512, 1),
            CommonBlock(512, 512, 1)
        )
        self.convP = nn.Conv2d(512, 512, 7, 1, bias=False)
        self.convP.weight = nn.Parameter(torch.ones(512, 512, 7, 7) / 49)
        self.bnP = nn.BatchNorm2d(512)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(512, classes_num)

    def forward(self, x,targets):
        x = self.prepare(x)
        x = self.layer1(x)
        x = self.layer2(x)    
        x = self.layer3(x)
        x = self.layer4(x)
        MC_loss = supervisor(x, targets, height=7, cnum=3)
        x = self.convP(x)
        x = self.bnP(x)
        x = F.relu(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        # print(x.size())
        return x,MC_loss

In [25]:
def main():  # dataset_1 dataset-resized
    root = '/content/dataset-resized'
    train_data = Trash(root, transform=True, train=True)
    val_data = Trash(root, train=False)
    train_loader = DataLoader(
        train_data,
        batch_size=64,
        shuffle=True,
        num_workers=0
    )
    val_loader = DataLoader(
        val_data,
        batch_size=64,
        shuffle=True,
        num_workers=0
    )
    model = ResNet34(6)
    print(model)
    device = torch.device('cuda:0')
    # state_dict = torch.load('/content/drive/MyDrive/myCal/task_2/model/CNN_sa.pkl')
    # model.load_state_dict(state_dict)
    model = model.to(device)
    criteon = nn.CrossEntropyLoss().to(device)  # 交叉熵损失
    # criteon = nn.MSELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-2)  # 优化器 a
    scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[40,80,100], gamma=0.1)
    for epoch in range(100):
      model.train()
      since = time.time()
      for x, label in train_loader:
          x, label = x.to(device), label.to(device)  # 传给GPU
          out ,MC_loss= model(x,label)  # 返回值
          loss = criteon(out, label)
          loss = loss + 1.5 * MC_loss[0] + 20 * MC_loss[1]
          optimizer.zero_grad()  # 梯度清零
          loss.backward()
          optimizer.step()
      print('Epoch: ', epoch, '训练集 loss: ', loss.item())  # item是转成数字
      model.eval()
      with torch.no_grad():
      # 测试集
        total_correct = 0
        total_num = 0
        for x, label in val_loader:
            x, label = x.to(device), label.to(device)
            out, MC_loss = model(x,label)  # 返回值
            loss = criteon(out, label)
            loss = loss + 1.2 * MC_loss[0] + 20 * MC_loss[1]
            pred = out.argmax(dim=1)  # 获得每行最大值列号
            correct = torch.eq(pred, label).float().sum().item()
            total_correct += correct
            total_num += x.size(0)

        acc = total_correct / total_num

        time_elapsed = time.time() - since
        print('Epoch: ', epoch, '测试集 loss: ', loss.item())  # item是转成数字
        print('Epoch: ', epoch,
              ' Training complete in {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60))
        timeCost = 'Training time {:.0f}min {:.0f}seconds'.format(time_elapsed // 60, time_elapsed % 60)
        print('Epoch: ', epoch, ' test acc: ', acc)
      scheduler.step()
#torch.save(model.state_dict(), '/content/drive/MyDrive/myCal/task_2/model/CNN_SA.pkl')




In [26]:
if __name__ == '__main__':
    main()

ResNet34(
  (prepare): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): CommonBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): CommonBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stri

KeyboardInterrupt: ignored